In [17]:
import copy

"""Import all the relevant libraries."""

from mkd_blockchain import MKDBlockchain
from blockchain_utils import BlockchainUtils
import block
from pubsub import pub
from node import Node
from transaction import Transaction
from sensor_transaction import SensorTransaction
from transaction_pool import TransactionPool
from wallet import Wallet
from Crypto.PublicKey import RSA
import pprint
import random
import time
from cluster import Cluster
import kdtree

In [18]:
"""Initialize Nodes with Genesis Block"""

print('START')
test_num = 1
MAX_NODES = 9999
num_nodes = 10
num_clusters = 2
forging_interval = 10  # Time interval.  10 units of time
nodes = [] #list of all nodes
blockchain_dimensions = 4
genesis_node_id = int(num_nodes/2)

#itterate through and initialize each node
then = time.time()
for i in range(num_nodes):
    new_node = Node(test_num, i+1, f'{test_num}.c0')
    #new_node.blockchain = mkd_blockchain
    new_node.start_listener(f'{test_num}.c0')
    nodes.append(new_node)
now = time.time()-then
print(f'block init time @ {num_nodes} nodes: {now}')

#  initialize each cluster
clusters = []
for i in range(num_clusters):
    new_cluster = Cluster(i+1, test_num)
    new_cluster.start_listener(f'{test_num}.c{i+1}')
    clusters.append(new_cluster)
    print(f'cluster {new_cluster.cluster_id} initialized')

genesis_forger = nodes[genesis_node_id-1].wallet.public_key_string()
mkd_blockchain = MKDBlockchain(blockchain_dimensions, genesis_node_id, genesis_forger)


publisher = nodes[genesis_node_id-1]

start_broadcast = time.time()
print('here')
publisher.publish(mkd_blockchain)
completed_broadcast = time.time() - start_broadcast
print(f'time to complete broadcast to {num_nodes} nodes: {completed_broadcast}')


START
block init time @ 10 nodes: 24.183109760284424
cluster 1 initialized
cluster 2 initialized
here
time to complete broadcast to 10 nodes: 0.003000020980834961


In [19]:
'''Create randomized list of node indexes'''

randomized_node_indexes = []
for i in range(num_nodes):
    randomized_node_indexes.append(i)
randomized_node_indexes: random.shuffle(randomized_node_indexes)
print(randomized_node_indexes)

[3, 4, 8, 9, 5, 7, 1, 2, 0, 6]


In [20]:
nodes_available = len(nodes)
while nodes_available != 0:
    amount_to_move = random.randint(0,2)
    random_cluster = f'{test_num}.c{(random.randint(1,num_clusters))}'
    print(f'nodes_available: {nodes_available} amount_to_move: {amount_to_move}')
    if nodes_available > 0 and nodes_available >= amount_to_move and nodes_available != 0:  # min 1 max 4
        for node in range(amount_to_move):
            nodes_available -= 1
            move_index = randomized_node_indexes.pop(0)
            print(f'move_index: {move_index}')
            nodes[move_index].move_node(nodes[move_index].cluster_id, random_cluster)

for i in range(len(clusters)):
    print(clusters[i].member_nodes)


nodes_available: 10 amount_to_move: 0
nodes_available: 10 amount_to_move: 2
move_index: 3
move_index: 4
nodes_available: 8 amount_to_move: 1
move_index: 8
nodes_available: 7 amount_to_move: 2
move_index: 9
move_index: 5
nodes_available: 5 amount_to_move: 2
move_index: 7
move_index: 1
nodes_available: 3 amount_to_move: 0
nodes_available: 3 amount_to_move: 2
move_index: 2
move_index: 0
nodes_available: 1 amount_to_move: 1
move_index: 6
[<node.Node object at 0x000002CA684EFDF0>, <node.Node object at 0x000002CA6804FCD0>, <node.Node object at 0x000002CA65ECEA60>, <node.Node object at 0x000002CA68045BB0>]
[<node.Node object at 0x000002CA6804F310>, <node.Node object at 0x000002CA6804F730>, <node.Node object at 0x000002CA684EF730>, <node.Node object at 0x000002CA684EF5E0>, <node.Node object at 0x000002CA68045CD0>, <node.Node object at 0x000002CA6804FEE0>]


In [21]:
"""Attempting to add a second block to the mkdtree_blockchain"""

number_of_forges = 4

for i in range(number_of_forges):
    random_node = random.randint(0, num_nodes-1)
    random_move = random.randint(0, num_nodes-1)
    if 1 == random.randint(0,5):  # to cause random movement
    #     random_move.node.move_node()
    forge_node = nodes[random_node]
    forge_node.mkd_forge()

for node in nodes:
    print(f'Cluster_id: {node.cluster_id} Node_id: {node.node_id} Tree size: {node.blockchain_size} Left size: {node.blockchain.blocks.left_size} Right size: {node.blockchain.blocks.right_size}')

Cluster_id: 1.c1 Node_id: 1 Tree size: 8 Left size: 2 Right size: 5
Cluster_id: 1.c2 Node_id: 2 Tree size: 7 Left size: 3 Right size: 3
Cluster_id: 1.c1 Node_id: 3 Tree size: 8 Left size: 2 Right size: 5
Cluster_id: 1.c2 Node_id: 4 Tree size: 7 Left size: 3 Right size: 3
Cluster_id: 1.c2 Node_id: 5 Tree size: 7 Left size: 3 Right size: 3
Cluster_id: 1.c1 Node_id: 6 Tree size: 8 Left size: 2 Right size: 5
Cluster_id: 1.c2 Node_id: 7 Tree size: 7 Left size: 3 Right size: 3
Cluster_id: 1.c2 Node_id: 8 Tree size: 7 Left size: 3 Right size: 3
Cluster_id: 1.c2 Node_id: 9 Tree size: 7 Left size: 3 Right size: 3
Cluster_id: 1.c1 Node_id: 10 Tree size: 8 Left size: 2 Right size: 5
